In [70]:
!pip install plotly==5.19.0 plotly --upgrade chart_studio

In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
import seaborn as sns
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
%matplotlib inline

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

import plotly.graph_objects as go

warnings.filterwarnings('ignore')

In [72]:
df =pd.read_csv('/kaggle/input/restaurant-dataset/Dataset .csv')

In [73]:
df

Restaurant ID           Restaurant Name  Country Code              City  \
0           6317637          Le Petit Souffle           162       Makati City   
1           6304287          Izakaya Kikufuji           162       Makati City   
2           6300002    Heat - Edsa Shangri-La           162  Mandaluyong City   
3           6318506                      Ooma           162  Mandaluyong City   
4           6314302               Sambo Kojin           162  Mandaluyong City   
...             ...                       ...           ...               ...   
9546        5915730               Naml۱ Gurme           208         ��stanbul   
9547        5908749              Ceviz A��ac۱           208         ��stanbul   
9548        5915807                     Huqqa           208         ��stanbul   
9549        5916112               A���k Kahve           208         ��stanbul   
9550        5927402  Walter's Coffee Roastery           208         ��stanbul   

                                                Address  \
0     Third Floor, Century City Mall, Kalayaan Avenu...   
1     Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2     Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3     Third Floor, Mega Fashion Hall, SM Megamall, O...   
4     Third Floor, Mega Atrium, SM Megamall, Ortigas...   
...                                                 ...   
9546  Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...   
9547  Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...   
9548  Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...   
9549  Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...   
9550  Cafea��a Mahallesi, Bademalt۱ Sokak, No 21/B, ...   

                                        Locality  \
0      Century City Mall, Poblacion, Makati City   
1     Little Tokyo, Legaspi Village, Makati City   
2     Edsa Shangri-La, Ortigas, Mandaluyong City   
3         SM Megamall, Ortigas, Mandaluyong City   
4         SM Megamall, Ortigas, Mandaluyong City   
...                                          ...   
9546                                    Karak�_y   
9547                                   Ko��uyolu   
9548                                 Kuru�_e��me   
9549                                 Kuru�_e��me   
9550                                        Moda   

                                       Locality Verbose   Longitude  \
0     Century City Mall, Poblacion, Makati City, Mak...  121.027535   
1     Little Tokyo, Legaspi Village, Makati City, Ma...  121.014101   
2     Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...  121.056831   
3     SM Megamall, Ortigas, Mandaluyong City, Mandal...  121.056475   
4     SM Megamall, Ortigas, Mandaluyong City, Mandal...  121.057508   
...                                                 ...         ...   
9546                                Karak�_y, ��stanbul   28.977392   
9547                               Ko��uyolu, ��stanbul   29.041297   
9548                             Kuru�_e��me, ��stanbul   29.034640   
9549                             Kuru�_e��me, ��stanbul   29.036019   
9550                                    Moda, ��stanbul   29.026016   

       Latitude                          Cuisines  ...          Currency  \
0     14.565443        French, Japanese, Desserts  ...  Botswana Pula(P)   
1     14.553708                          Japanese  ...  Botswana Pula(P)   
2     14.581404  Seafood, Asian, Filipino, Indian  ...  Botswana Pula(P)   
3     14.585318                   Japanese, Sushi  ...  Botswana Pula(P)   
4     14.584450                  Japanese, Korean  ...  Botswana Pula(P)   
...         ...                               ...  ...               ...   
9546  41.022793                           Turkish  ...  Turkish Lira(TL)   
9547  41.009847   World Cuisine, Patisserie, Cafe  ...  Turkish Lira(TL)   
9548  41.055817            Italian, World Cuisine  ...  Turkish Lira(TL)   
9549  41.057979                   Restaurant Cafe  ...  Turkish Lira(TL)   
9550  40.9

In [74]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.shape

(9551, 21)

In [75]:
df['Cuisines'].fillna(df['Cuisines'].mode()[0], inplace=True)


In [76]:
df.columns.to_list()

['Restaurant ID',
 'Restaurant Name',
 'Country Code',
 'City',
 'Address',
 'Locality',
 'Locality Verbose',
 'Longitude',
 'Latitude',
 'Cuisines',
 'Average Cost for two',
 'Currency',
 'Has Table booking',
 'Has Online delivery',
 'Is delivering now',
 'Switch to order menu',
 'Price range',
 'Aggregate rating',
 'Rating color',
 'Rating text',
 'Votes']

In [77]:
# Preprocessing: Split multiple cuisines into lists
df['Cuisine_List'] = df['Cuisines'].str.split(', ')

In [78]:
# Flatten the list of cuisines (for clustering)
flat_cuisines = [cuisine for sublist in df['Cuisine_List'] for cuisine in sublist]

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Step 1: Use TF-IDF to convert cuisines to vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(flat_cuisines)

In [80]:
# Step 2: Apply KMeans clustering (or another clustering algorithm)
num_clusters = 5  # Adjust based on your needs
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)

KMeans(n_clusters=5, random_state=42)

In [81]:
# Step 3: Create a mapping from each cuisine to its cluster label
clusters = kmeans.predict(X)
cluster_map = {flat_cuisines[i]: clusters[i] for i in range(len(flat_cuisines))}


In [82]:
cluster_names = {
    0: 'Asian',
    1: 'European',
    2: 'Desserts',
    3: 'Cafe',
    4: 'World Cuisine'
}

# Step 4: Apply the cluster mapping to the original dataset
df['Mapped_Cuisines'] = df['Cuisine_List'].apply(lambda x: ', '.join([cluster_names[cluster_map[c]] for c in x]))


In [83]:
# Function to remove duplicates within each row of Mapped_Cuisines
def remove_duplicates(cuisine_string):
    # Split the cuisines by ', ' to create a list
    cuisine_list = cuisine_string.split(', ')
    # Remove duplicates by converting the list to a set and back to a list, then join it back into a string
    unique_cuisines = ', '.join(sorted(set(cuisine_list), key=cuisine_list.index))  # Maintain order
    return unique_cuisines

# Apply the function to the 'Mapped_Cuisines' column
df['Mapped_Cuisines'] = df['Mapped_Cuisines'].apply(remove_duplicates)

# Display the result
print(df[['Restaurant Name', 'Cuisines', 'Mapped_Cuisines']].head())


          Restaurant Name                          Cuisines  Mapped_Cuisines
0        Le Petit Souffle        French, Japanese, Desserts            Asian
1        Izakaya Kikufuji                          Japanese            Asian
2  Heat - Edsa Shangri-La  Seafood, Asian, Filipino, Indian  Asian, European
3                    Ooma                   Japanese, Sushi            Asian
4             Sambo Kojin                  Japanese, Korean            Asian


In [129]:
# Define a dictionary to map various combinations to 5 broad categories
category_map = {
    'Asian': 'Asian',
    'European': 'European',
    'Cafe': 'Cafe',
    'Desserts': 'Desserts',
    'World Cuisine': 'World Cuisine',
    'Asian, Desserts': 'Asian',
    'European, Cafe': 'European',
    'European, Asian': 'European',
    'Desserts, Asian': 'Asian',
    'Asian, World Cuisine': 'World Cuisine',
    'European, Desserts': 'European',
    'Cafe, Asian': 'Cafe',
    'Cafe, European': 'Cafe',
    'Cafe, Desserts': 'Cafe',
    'European, Cafe, Desserts': 'European',
    'European, Cafe, Asian': 'World Cuisine',
    'European, Asian, Cafe':'World Cuisine',
    'Asian, European':'World Cuisine',
    'Desserts':'European',
    'Asian, Cafe':'Asian',
    'Cafe':'Asian',
    'World Cuisine, Asian':'World Cuisine',
    'Asian, European, Cafe':'World Cuisine',
    'Desserts, European':'European',
    'Asian, European, Desserts':'World Cuisine',
    # Add any additional combinations here
}

# Function to map the cuisines to the 5 categories
def map_to_broad_category(cuisine_string):
    # If the combination is in the dictionary, map it, else return as is (for unexpected values)
    return category_map.get(cuisine_string, cuisine_string)

# Apply the mapping to the 'Mapped_Cuisines' column
df['Mapped_Cuisines'] = df['Mapped_Cuisines'].apply(map_to_broad_category)

# Display the updated counts for each category
print(df['Mapped_Cuisines'].value_counts())


Mapped_Cuisines
Asian                                             4513
European                                          3192
World Cuisine                                      987
Desserts, Cafe                                      48
World Cuisine, Asian, Desserts                      37
                                                  ... 
Asian, Desserts, Cafe, European                      1
Desserts, World Cuisine, Cafe                        1
Asian, Cafe, Desserts, World Cuisine                 1
Desserts, Asian, World Cuisine, European             1
Desserts, Asian, World Cuisine, European, Cafe       1
Name: count, Length: 114, dtype: int64


In [130]:
Cuisines = df['Mapped_Cuisines'].value_counts()
Cuisines

Mapped_Cuisines
Asian                                             4513
European                                          3192
World Cuisine                                      987
Desserts, Cafe                                      48
World Cuisine, Asian, Desserts                      37
                                                  ... 
Asian, Desserts, Cafe, European                      1
Desserts, World Cuisine, Cafe                        1
Asian, Cafe, Desserts, World Cuisine                 1
Desserts, Asian, World Cuisine, European             1
Desserts, Asian, World Cuisine, European, Cafe       1
Name: count, Length: 114, dtype: int64

In [131]:
Cuisines_m = Cuisines[Cuisines > 100]
Cuisines_m

Mapped_Cuisines
Asian            4513
European         3192
World Cuisine     987
Name: count, dtype: int64

In [132]:
to_keep = Cuisines[Cuisines >= 100].index
df_cleaned = df[df['Mapped_Cuisines'].isin(to_keep)]


In [133]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8692 entries, 0 to 9550
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         8692 non-null   int64  
 1   Restaurant Name       8692 non-null   object 
 2   Country Code          8692 non-null   int64  
 3   City                  8692 non-null   object 
 4   Address               8692 non-null   object 
 5   Locality              8692 non-null   object 
 6   Locality Verbose      8692 non-null   object 
 7   Longitude             8692 non-null   float64
 8   Latitude              8692 non-null   float64
 9   Cuisines              8692 non-null   object 
 10  Average Cost for two  8692 non-null   int64  
 11  Currency              8692 non-null   object 
 12  Has Table booking     8692 non-null   object 
 13  Has Online delivery   8692 non-null   object 
 14  Is delivering now     8692 non-null   object 
 15  Switch to order menu  8692

In [150]:
X = df_cleaned.drop(columns=['Mapped_Cuisines','Cuisines','Cuisine_List','Restaurant ID','Restaurant Name','Country Code','Locality','Latitude','Longitude','Locality Verbose','Address'])  # Drop the target variable from features
y = df_cleaned['Mapped_Cuisines']  # The target variable you want to predict

In [151]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8692 entries, 0 to 9550
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   City                  8692 non-null   object 
 1   Average Cost for two  8692 non-null   int64  
 2   Currency              8692 non-null   object 
 3   Has Table booking     8692 non-null   object 
 4   Has Online delivery   8692 non-null   object 
 5   Is delivering now     8692 non-null   object 
 6   Switch to order menu  8692 non-null   object 
 7   Price range           8692 non-null   int64  
 8   Aggregate rating      8692 non-null   float64
 9   Rating color          8692 non-null   object 
 10  Rating text           8692 non-null   object 
 11  Votes                 8692 non-null   int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 882.8+ KB


In [156]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Encoding other categorical variables like 'City', 'Has Table booking', 'Has Online delivery', 'Is delivering now'
X['City'] = label_encoder.fit_transform(X['City'])
X['Has Table booking'] = label_encoder.fit_transform(X['Has Table booking'])
X['Has Online delivery'] = label_encoder.fit_transform(X['Has Online delivery'])
X['Is delivering now'] = label_encoder.fit_transform(X['Is delivering now'])
X['Currency'] = label_encoder.fit_transform(X['Currency'])
X['Switch to order menu'] = label_encoder.fit_transform(X['Switch to order menu'])

X['Rating color'] = label_encoder.fit_transform(X['Rating color'])
X['Rating text'] = label_encoder.fit_transform(X['Rating text'])

In [176]:
# One-Hot Encode the target
y_encoded = pd.get_dummies(y)

In [158]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8692 entries, 0 to 9550
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   City                  8692 non-null   int64  
 1   Average Cost for two  8692 non-null   int64  
 2   Currency              8692 non-null   int64  
 3   Has Table booking     8692 non-null   int64  
 4   Has Online delivery   8692 non-null   int64  
 5   Is delivering now     8692 non-null   int64  
 6   Switch to order menu  8692 non-null   int64  
 7   Price range           8692 non-null   int64  
 8   Aggregate rating      8692 non-null   float64
 9   Rating color          8692 non-null   int64  
 10  Rating text           8692 non-null   int64  
 11  Votes                 8692 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 882.8 KB


In [179]:
X_train.shape

(6953, 12)

In [180]:
y_train.shape

(6953,)

In [159]:
# Step 7: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Model Training
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 9: Model Evaluation
y_pred = clf.predict(X_test)

# Accuracy and Classification Report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Accuracy: 54.80%
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.69      0.65       935
           1       0.47      0.46      0.46       624
           2       0.27      0.14      0.19       180

    accuracy                           0.55      1739
   macro avg       0.45      0.43      0.43      1739
weighted avg       0.53      0.55      0.54      1739



In [200]:
#CatBoostClassifier THE BEST MODEL 
from catboost import CatBoostClassifier
model = CatBoostClassifier()

model.fit(X_train,y_train)
# Make predictions on new data
y_pred = model.predict(X_test)

# Output predictions
#print("Predictions:", y_pred)

from sklearn.metrics import accuracy_score

# Assuming you have true labels for the test set


# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Learning rate set to 0.087352
0:	learn: 1.0711546	total: 5.6ms	remaining: 5.59s
1:	learn: 1.0461853	total: 10.1ms	remaining: 5.03s
2:	learn: 1.0264406	total: 13.1ms	remaining: 4.36s
3:	learn: 1.0084767	total: 16.6ms	remaining: 4.12s
4:	learn: 0.9928098	total: 20.7ms	remaining: 4.12s
5:	learn: 0.9789695	total: 24.7ms	remaining: 4.08s
6:	learn: 0.9665143	total: 29.3ms	remaining: 4.16s
7:	learn: 0.9562680	total: 33.5ms	remaining: 4.16s
8:	learn: 0.9473955	total: 37.7ms	remaining: 4.16s
9:	learn: 0.9394680	total: 41.8ms	remaining: 4.14s
10:	learn: 0.9332383	total: 46ms	remaining: 4.13s
11:	learn: 0.9267364	total: 49.9ms	remaining: 4.11s
12:	learn: 0.9213485	total: 53.9ms	remaining: 4.09s
13:	learn: 0.9158186	total: 58.1ms	remaining: 4.09s
14:	learn: 0.9107939	total: 62.3ms	remaining: 4.09s
15:	learn: 0.9062957	total: 66.5ms	remaining: 4.09s
16:	learn: 0.9031803	total: 70.7ms	remaining: 4.09s
17:	learn: 0.8989802	total: 74.9ms	remaining: 4.08s
18:	learn: 0.8960016	total: 79ms	remaining: 4.0

In [201]:
from xgboost import XGBClassifier
model = XGBClassifier()

model.fit(X_train,y_train)
# Make predictions on new data
y_pred = model.predict(X_test)

# Output predictions
print("Predictions:", y_pred)

from sklearn.metrics import accuracy_score

# Assuming you have true labels for the test set


# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Predictions: [0 0 0 ... 1 0 0]
Accuracy: 0.5646923519263944
Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.69      0.66       935
           1       0.49      0.50      0.50       624
           2       0.33      0.13      0.18       180

    accuracy                           0.56      1739
   macro avg       0.48      0.44      0.45      1739
weighted avg       0.55      0.56      0.55      1739



In [202]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X_train,y_train)
# Make predictions on new data
y_pred = model.predict(X_test)

# Output predictions
print("Predictions:", y_pred)

from sklearn.metrics import accuracy_score

# Assuming you have true labels for the test set


# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Predictions: [0 0 0 ... 1 0 0]
Accuracy: 0.5796434732604946
Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.74      0.68       935
           1       0.50      0.48      0.49       624
           2       0.41      0.08      0.13       180

    accuracy                           0.58      1739
   macro avg       0.51      0.43      0.43      1739
weighted avg       0.56      0.58      0.56      1739



In [197]:
# 2. Split the data into training and test sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [198]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import numpy as np

# Assuming X_train1 and X_test1 are already scaled and ready for input

# Reshape the data to 3D (samples, timesteps, features) for LSTM
# Here, timesteps = 1 as we are treating each sample as a single sequence
X_train1 = np.reshape(X_train1, (X_train1.shape[0], 1, X_train1.shape[1]))
X_test1 = np.reshape(X_test1, (X_test1.shape[0], 1, X_test1.shape[1]))

# 3. Build the ANN + LSTM Model
model = Sequential()

# Add LSTM Layer (128 units) - first layer in the model
model.add(LSTM(128, input_shape=(X_train1.shape[1], X_train1.shape[2]), return_sequences=False))

# Optionally add dropout to prevent overfitting
model.add(Dropout(0.1))

# Hidden layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output layer (number of classes in y_encoded)
model.add(Dense(y_encoded.shape[1], activation='softmax'))

# 4. Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 5. Train the model
model.fit(X_train1, y_train1, epochs=100, batch_size=32, validation_data=(X_test1, y_test1))

# 6. Evaluate the model
loss, accuracy = model.evaluate(X_test1, y_test1)
print(f"Test Accuracy: {accuracy*100:.2f}%")

# 7. Make predictions (optional)
predictions = model.predict(X_test1)


Epoch 1/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.4997 - loss: 0.9435 - val_accuracy: 0.5273 - val_loss: 0.8834
Epoch 2/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5083 - loss: 0.9165 - val_accuracy: 0.5446 - val_loss: 0.8893
Epoch 3/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5112 - loss: 0.9153 - val_accuracy: 0.5474 - val_loss: 0.8845
Epoch 4/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5130 - loss: 0.9121 - val_accuracy: 0.5463 - val_loss: 0.8774
Epoch 5/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5188 - loss: 0.9114 - val_accuracy: 0.5313 - val_loss: 0.8834
Epoch 6/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5129 - loss: 0.9005 - val_accuracy: 0.5371 - val_loss: 0.8856
Epoch 7/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5198 - loss: 0.9121 - val_accuracy: 0.5348 - val_loss: 0.8781
Epoch 8/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5227 - loss: 0.9072 - val_accu